In [6]:
import requests
import asyncio
import aiohttp
import copy
from datetime import datetime
from db_connector import DB
from logger import logging

Task_URL = "https://api.apify.com/v2/actor-tasks/yellow_saint~instagram-scraper-task/input?token=apify_api_qIc4Lwctqfr2WAFjmyQZ2mwd1zYhwO0JFLvE"
Scrape_Link_dataset = "https://api.apify.com/v2/actor-tasks/yellow_saint~instagram-scraper-task/run-sync-get-dataset-items?token=apify_api_qIc4Lwctqfr2WAFjmyQZ2mwd1zYhwO0JFLvE"
Insta_Link = 'https://www.instagram.com/{x}/'
inputs  =  requests.get(Task_URL)



#apify_client = ApifyClient(token)

# Start an actor and wait for it to finish
#task_call = apify_client.task('yellow_saint~instagram-scraper-task').call(task_input=obj_list[0])

# Fetch results from the actor run's default dataset
#dataset_items = apify_client.dataset(task_call['defaultDatasetId']).list_items().items

In [32]:
obj = inputs.json()


In [1]:
class Scrape():
    Task_URL = "https://api.apify.com/v2/actor-tasks/yellow_saint~instagram-scraper-task/input?token=apify_api_qIc4Lwctqfr2WAFjmyQZ2mwd1zYhwO0JFLvE"
    Scrape_Link_dataset = "https://api.apify.com/v2/actor-tasks/yellow_saint~instagram-scraper-task/run-sync-get-dataset-items?token=apify_api_qIc4Lwctqfr2WAFjmyQZ2mwd1zYhwO0JFLvE"
    Insta_Link = 'https://www.instagram.com/{x}/'
    inputs  =  requests.get(Task_URL)


    def __init__(self) -> None:
        self.inputs  =  requests.get(self.ask_URL)
        self.DB = DB()
        logging.info("DB connected at Scraper")
        self.To_scan()
        
    def To_scan(self) -> list:
        Unsorted_list = self.DB.pull(db = "Clean",document="To_scan",filter= {},colunm={"_id":0})
        sorted_data = sorted(Unsorted_list, key=lambda x: x['priority'])
        final_list = []
        for i in sorted_data:
            final_list.append(i["id"])
        self.To_scan = final_list    
        logging.info("To_scan listed Created")

    
    async def get(
        session: aiohttp.ClientSession,
        input_obj:dict,
        url : str
        ) -> dict:

        print(f"Requesting {url}")
        resp = await session.request('POST', url=url,json=input_obj)

        # Note that this may raise an exception for non-2xx responses
        # You can either handle that here, or pass the exception through
        data = await resp.json()
        print(f"Received data for {url}")
        return data


    async def main(self,input_objs,url, **kwargs):
        # Asynchronous context manager.  Prefer this rather
        # than using a different session for each GET request
        async with aiohttp.ClientSession() as session:
            tasks = []
            for obj in input_objs:
                tasks.append(self.get(session=session, url=url,input_obj=obj, **kwargs))
            # asyncio.gather() will wait on the entire task set to be
            # completed.  If you want to process results greedily as they come in,
            # loop over asyncio.as_completed()
            data = await asyncio.gather(*tasks, return_exceptions=True)
            return data
    def Cleaner(self,data) -> None:
        """
            The Scrape function produces a data in the form of 2-d nested Lists -> list[list] 
            To pull the lists out of the main list we use 2 loops to pull them out
            can be done with recrusion but will try later,
            Also traverse the data to find more related profiles to further scan       
        """
        cleaned_data = []
        for i in data:
            for j in i:
                cleaned_data.append(j)
        
        for i in cleaned_data:

            for j in i["relatedProfiles"]:
                to_scan_dict = {"id":j["username"],"priority":2}
                self.DB.push(to_scan_dict,"Clean","To_scan")
            i["Date"] = datetime.date   
            self.DB.push(i,"Raw","Creators")        
    

    def input_obj_maker(List_of_url:list,No_of_acc:int) -> list:

        obj_list = []
        for i in  range(0,len(List_of_url)-No_of_acc,No_of_acc):
            temp = copy.deepcopy(obj)
            temp["directUrls"] =  List_of_url[i:i+No_of_acc]
            obj_list.append(temp)
        return  obj_list  

    async def scrape(self):
        data = await self.main( self.To_scan,self.Scrape_Link_dataset)
        
        return self.Cleaner(data) 
        

SyntaxError: invalid syntax (3265431808.py, line 72)

In [34]:
async def get(
    session: aiohttp.ClientSession,
    input_obj:dict,
    url : str
) -> dict:
    print(f"Requesting {url}")
    resp = await session.request('POST', url=url,json=input_obj)

    # Note that this may raise an exception for non-2xx responses
    # You can either handle that here, or pass the exception through
    data = await resp.json()
    print(f"Received data for {url}")
    return data
async def main(input_objs,url, **kwargs):
    # Asynchronous context manager.  Prefer this rather
    # than using a different session for each GET request
    async with aiohttp.ClientSession() as session:
        tasks = []
        for obj in input_objs:
            tasks.append(get(session=session, url=url,input_obj=obj, **kwargs))
        # asyncio.gather() will wait on the entire task set to be
        # completed.  If you want to process results greedily as they come in,
        # loop over asyncio.as_completed()
        data = await asyncio.gather(*tasks, return_exceptions=True)
        return data


In [53]:
obj = Scrape()

In [54]:
obj.To_scan

[545, 54, 54545]

In [44]:

data = obj.To_scan
for i in data:
    data.append(i)
    if len(data) == 20:
        break
data

[{'id': 54545, 'priority': 3},
 {'id': 545, 'priority': 2},
 {'id': 545, 'priority': 1},
 {'id': 54545, 'priority': 3},
 {'id': 545, 'priority': 2},
 {'id': 545, 'priority': 1},
 {'id': 54545, 'priority': 3},
 {'id': 545, 'priority': 2},
 {'id': 545, 'priority': 1},
 {'id': 54545, 'priority': 3},
 {'id': 545, 'priority': 2},
 {'id': 545, 'priority': 1},
 {'id': 54545, 'priority': 3},
 {'id': 545, 'priority': 2},
 {'id': 545, 'priority': 1},
 {'id': 54545, 'priority': 3},
 {'id': 545, 'priority': 2},
 {'id': 545, 'priority': 1},
 {'id': 54545, 'priority': 3},
 {'id': 545, 'priority': 2}]

{'id': 545, 'priority': 1}
{'id': 545, 'priority': 1}
{'id': 545, 'priority': 1}
{'id': 545, 'priority': 1}
{'id': 545, 'priority': 1}
{'id': 545, 'priority': 1}
{'id': 545, 'priority': 2}
{'id': 545, 'priority': 2}
{'id': 545, 'priority': 2}
{'id': 545, 'priority': 2}
{'id': 545, 'priority': 2}
{'id': 545, 'priority': 2}
{'id': 545, 'priority': 2}
{'id': 54545, 'priority': 3}
{'id': 54545, 'priority': 3}
{'id': 54545, 'priority': 3}
{'id': 54545, 'priority': 3}
{'id': 54545, 'priority': 3}
{'id': 54545, 'priority': 3}
{'id': 54545, 'priority': 3}


In [173]:
data_2 = []
for i in data:
    for j in i:
        data_2.append(j)
to_scan = []
for i in data_2:
    for j in i["relatedProfiles"]:
        to_scan.append(Insta_Link.format(x = j["username"]))

In [180]:
obj_list = input_obj_maker(to_scan,10)

In [182]:
len(obj_list)

30

In [184]:
len(to_scan)

309